# First Modeling (Content)

In [1]:
import numpy as np 
import os 
import pandas as pd 
import re
import string
import seaborn as sns
from rake_nltk import Rake
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import vstack

In [ ]:
#%pip install rake-nltk

In [29]:
df_anime = pd.read_csv("../data/df_anime_export.csv")
df_profile = pd.read_csv("../data/df_profile_export.csv")
df_review = pd.read_csv("../data/df_review_export.csv")
df_an_pro_rev = pd.read_csv("../data/merged_rev_df_export.csv")
df_an_pro = pd.read_csv("../data/merged_df_export.csv")

In [3]:
print(f"Len of all 3 df: {len(df_an_pro_rev)}, len of anime+profiles: {len(df_an_pro)}")

Len of all 3 df: 27299, len of anime+profiles: 216297


In [38]:
df_an_pro_rev.head(10)

,Unnamed: 0,uid,title,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,profile,gender,birthday,review_score,review_score_indepth,text_preprocessed
0,0,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",Drevorn,Male,"Mar 20, 1995",10,"{'Overall': '10', 'Story': '10', 'Animation': ...",2nd season haikyuu ! ! pick right 1st season e...
1,1,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",Drevorn,Male,"Mar 20, 1995",10,"{'Overall': '10', 'Story': '10', 'Animation': ...",2nd season haikyuu ! ! pick right 1st season e...
2,2,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",NICK1994,Male,"Nov 13, 1994",10,"{'Overall': '10', 'Story': '10', 'Animation': ...",intense ! ! ! ! ! ! word describes plot season...
3,3,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",NICK1994,Male,"Nov 13, 1994",10,"{'Overall': '10', 'Story': '10', 'Animation': ...",intense ! ! ! ! ! ! word describes plot season...
4,4,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",RuiAizawa,Female,"Aug 24, 1997",10,"{'Overall': '10', 'Story': '10', 'Animation': ...","honestly , kind anime absolutely interest voll..."
5,5,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",RuiAizawa,Female,"Aug 24, 1997",10,"{'Overall': '10', 'Story': '10', 'Animation': ...","honestly , kind anime absolutely interest voll..."
6,6,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",Shinoa_,Female,Oct 31,10,"{'Overall': '10', 'Story': '9', 'Animation': '...",first review may actually sound biased anime ....
7,7,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",Shinoa_,Female,Oct 31,10,"{'Overall': '10', 'Story': '9', 'Animation': '...",first review may actually sound biased anime ....
8,8,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",tekkaadan,Male,"Feb 27, 1994",10,"{'Overall': '10', 'Story': '10', 'Animation': ...",“ second season ” lot anime series tends bring...
9,9,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",tekkaadan,Male,"Feb 27, 1994",10,"{'Overall': '10', 'Story': '10', 'Animation': ...",“ second season ” lot anime series tends bring...


## View resetter

In [25]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [36]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

## Cleaning df anime some more

In [ ]:
df_anime = pd.read_csv("../data/df_anime_export.csv")

In [32]:
df_anime.head(1)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed
0,0,28891,Haikyuu!! Second Season,"Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the Spring tournament instead. \r\n \r\nWhen they receive an invitation from long-standing rival Nekoma High, Karasuno agrees to take part in a large training camp alongside many notable volleyball teams in Tokyo and even some national level players. By playing with some of the toughest teams in Japan, they hope not only to sharpen their skills, but also come up with new attacks that would strengthen them. Moreover, Hinata and Kageyama attempt to devise a more powerful weapon, one that could possibly break the sturdiest of blocks. \r\n \r\nFacing what may be their last chance at victory before the senior players graduate, the members of Karasuno's volleyball team must learn to settle their differences and train harder than ever if they hope to overcome formidable opponents old and new—including their archrival Aoba Jousai and its world-class setter Tooru Oikawa. \r\n \r\n[Written by MAL Rewrite]","['Comedy', 'Sports', 'Drama', 'School', 'Shounen']","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno high school volleyball team attempt refocus effort , aiming conquer spring tournament instead . receive invitation long-standing rival nekoma high , karasuno agrees take part large training camp alongside many notable volleyball team tokyo even national level player . playing toughest team japan , hope sharpen skill , also come new attack would strengthen . moreover , hinata kageyama attempt devise powerful weapon , one could possibly break sturdiest block . facing may last chance victory senior player graduate , member karasuno 's volleyball team must learn settle difference train harder ever hope overcome formidable opponent old new—including archrival aoba jousai world-class setter tooru oikawa ."


In [33]:
for i, genre in enumerate(df_anime['genre']):
    df_anime.at[i, 'genre'] = genre.replace('Martial Arts', 'Martial_Arts')
    df_anime.at[i, 'genre'] = genre.replace('Super Power', 'Super_Power')
    df_anime.at[i, 'genre'] = genre.replace('Slice of Life', 'Slice_of_Life')

In [34]:
df_anime['genre'].value_counts()

['Hentai']                                                                                                                                                   923
['Music']                                                                                                                                                    692
['Comedy']                                                                                                                                                   622
['Kids']                                                                                                                                                     404
['Dementia']                                                                                                                                                 201
['Slice_of_Life']                                                                                                                                            188
['Music', 'Kids']                 

In [35]:
#clean genre so it can be used in the rec system
for i, row_genre in enumerate(df_anime['genre']):
    row_genre = row_genre.replace('[','')
    row_genre = row_genre.replace(']','')
    row_genre = row_genre.replace("'","")
    row_genre = row_genre.replace(",","")
    df_anime['genre'].iloc[i] = row_genre
df_anime['genre']

F:\Anaconda\envs\learn-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0                                                                                       Comedy Sports Drama School Shounen
1                                                                                       Drama Music Romance School Shounen
2                                                                                   Sci-Fi Adventure Mystery Drama Fantasy
3                                                             Action Military Adventure Comedy Drama Magic Fantasy Shounen
4                                                                                      Action Mystery Supernatural Vampire
5                                                                                 Action Slice_of_Life Comedy Supernatural
6                                                                                             Adventure Supernatural Drama
7                                                                            Action Demons Historical Shounen Supernatural
8               

## Begin recommender system 1

In [ ]:
df = df_an_pro_rev

In [ ]:
syp = df['synopsis_processed']
genre = df['genre']
title = df['title']
#for i, t in enumerate(title):
#    title[i] = t.lower()

In [ ]:
#extracting keywords for recommender
rake = Rake()
keywords = []
for plot in syp:
    rake.extract_keywords_from_text(plot)
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    keywords.append(keywords_i_string)
df['keywords'] = keywords
df['keywords'][0]

In [ ]:
df['bag_words'] = df['genre'] + df['keywords']

In [ ]:
vectorizer = CountVectorizer()
vectorized_bag_of_words = vectorizer.fit_transform(df['bag_words'])
vectorized_bag_of_words = vectorized_bag_of_words.toarray()

In [ ]:
def recommend(show_title, n_recom):
    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[list(np.where(df["title"] == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title 
    similarity_dataframe =  similarity_dataframe.iloc[:,0]
    similarity_dataframe = similarity_dataframe.sort_values(ascending = False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()
    return list(similarity_dataframe.index)[1:n_recom + 1]

In [ ]:
recommend("Haikyuu!! Second Season",10)

## Begin rec system 2 using other method for synopsis

In [ ]:
df_2 = df_anime

In [ ]:
df_2.head(1)

In [ ]:
syp_2 = df_anime['synopsis']
genre_2 = df_anime['genre']
title_2 = df_anime['title']
#for i, t in enumerate(title_2):
#    title_2[i] = t.lower()
#clean genre so it can be used in the rec system
for i, row_genre in enumerate(genre_2):
    row_genre = row_genre.replace('[','')
    row_genre = row_genre.replace(']','')
    row_genre = row_genre.replace("'","")
    row_genre = row_genre.replace(",","")
    genre_2.iloc[i] = row_genre
genre_2

In [ ]:
#checking synopsis
syp_2[0]

In [ ]:
#extracting keywords for recommender
rake = Rake()
keywords = []
for plot in syp_2:
    rake.extract_keywords_from_text(plot)
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    keywords.append(keywords_i_string)
df_2['keywords'] = keywords
df_2['keywords'][0]    

In [ ]:
df_2['bag_words'] = df_2['genre'] + df_2['keywords']

In [ ]:
vectorizer_2 = CountVectorizer()
vectorized_bag_of_words_2 = vectorizer_2.fit_transform(df_2['bag_words'])
vectorized_bag_of_words_2 = vectorized_bag_of_words_2.toarray()

In [ ]:
def recommend_2(show_title, n_recom):
    similarity_matrix = cosine_similarity(vectorized_bag_of_words_2, vectorized_bag_of_words_2[list(np.where(df_2["title"] == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title_2
    similarity_dataframe =  similarity_dataframe.iloc[:,0]
    similarity_dataframe = similarity_dataframe.sort_values(ascending = False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()
    return list(similarity_dataframe.index)[1:n_recom + 1]

In [ ]:
recommend_2("Haikyuu!! Second Season",10)

### Some conclusions from this model:
- it would be pointless using df_an_pro_rev for this version of rec system, since it is just using synopsis and genre.
- Will make a 3rd model using only the preprocessed text too instead of everything.

## Model 3: final model for content based recommender

In [ ]:
df_anime.head(2)

In [ ]:
df_anime.drop(columns = (['keywords', 'bag_words']), inplace = True)

In [ ]:
df_3 = df_anime

In [ ]:
syp_3 = df_anime['synopsis_processed']
genre_3 = df_anime['genre']
title_3 = df_anime['title']
#no need to make title lowercase I believe
#for i, t in enumerate(title_3):
#    title_3[i] = t.lower()
#clean genre so it can be used in the rec system
for i, row_genre in enumerate(genre_3):
    row_genre = row_genre.replace('[','')
    row_genre = row_genre.replace(']','')
    row_genre = row_genre.replace("'","")
    row_genre = row_genre.replace(",","")
    genre_3.iloc[i] = row_genre
genre_3

In [ ]:
syp_3 = syp_3.astype(str)

In [ ]:
#extracting keywords for recommender
rake = Rake()
keywords = []
for plot in syp_3:
    rake.extract_keywords_from_text(plot)
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    keywords.append(keywords_i_string)
df_3['keywords'] = keywords
df_3['keywords'][0]

In [ ]:
df_3['bag_words'] = df_3['genre'] + df_3['keywords']

In [ ]:
vectorizer_3 = CountVectorizer()
vectorized_bag_of_words_3 = vectorizer_3.fit_transform(df_3['bag_words'])
vectorized_bag_of_words_3 = vectorized_bag_of_words_3.toarray()

def recommend_3(show_title, n_recom):
    similarity_matrix = cosine_similarity(vectorized_bag_of_words_3, vectorized_bag_of_words_3[list(np.where(df_3["title"] == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title_3
    similarity_dataframe =  similarity_dataframe.iloc[:,0]
    similarity_dataframe = similarity_dataframe.sort_values(ascending = False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()
    return list(similarity_dataframe.index)[1:n_recom + 1]

In [ ]:
recommend_3("Haikyuu!! Second Season",10)

In [ ]:
recommend("Haikyuu!! Second Season",10)

In [ ]:
recommend_2("Haikyuu!! Second Season",10)